In [ ]:
import os
from dotenv     import load_dotenv

load_dotenv()

In [ ]:
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [ ]:
from langchain_huggingface  import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

In [ ]:
from langchain_chroma                       import Chroma
from langchain_community.document_loaders   import WebBaseLoader
from langchain_core.prompts                 import ChatPromptTemplate
from langchain_text_splitters               import RecursiveCharacterTextSplitter
from langchain.chains                       import create_retrieval_chain
from langchain.chains.combine_documents     import create_stuff_documents_chain
import bs4

## Web Based Loader

In [ ]:
## Load, Chunk and Index the contents of the blog to create a retriever

webLoader = WebBaseLoader(
    web_paths  = ("https://lilianweng.github.io/posts/2023-06-23-agent",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    ),
)

In [ ]:
documents = webLoader.load()

In [ ]:
documents 

In [ ]:
textSplitter    = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits          = textSplitter.split_documents(documents)

In [ ]:
vectorStore = Chroma.from_documents(documents = splits, embedding = embeddings)
retriever = vectorStore.as_retriever()

In [ ]:
retriever

## Prompt Template

In [ ]:
systemPrompt = (
    "You are an assistant for question-answering tasks."
    "Use the following peices of retrieved context to answer "
    "the questions. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum to keep the "
    "answer concise. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", systemPrompt),
        ("human", "{input}")
    ]
)

In [ ]:
questionAnswerChain = create_stuff_documents_chain(llm, prompt)
ragChain            = create_retrieval_chain(retriever, questionAnswerChain)

In [ ]:
response = ragChain.invoke({"input": "What is Self-Reflection?"})

In [ ]:
response['answer']

In [ ]:
response = ragChain.invoke({"input": "How do we achieve it?"})
response['answer']

## Chat History

In [ ]:
from langchain.chains           import create_history_aware_retriever
from langchain_core.prompts     import MessagesPlaceholder

contextualizeSystemPrompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualizeQuestionPrompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualizeSystemPrompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

In [ ]:
historyAwareRetriever = create_history_aware_retriever(llm, retriever, contextualizeQuestionPrompt)
historyAwareRetriever

In [ ]:
questionAnswerPrompt = ChatPromptTemplate.from_messages(
    [
        ("system", systemPrompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

## Chain

In [ ]:
questionAnswerChain = create_stuff_documents_chain(llm, questionAnswerPrompt)
ragChain            = create_retrieval_chain(historyAwareRetriever, questionAnswerChain)

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage

chat_history=[]
question="What is Self-Reflection"
response1 = ragChain.invoke({"input":question, "chat_history" : chat_history})
print(response1["answer"])

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

In [ ]:
question2 = "Tell me more about it?"
response2 = ragChain.invoke({"input":question2, "chat_history" : chat_history})
print(response2['answer'])

In [ ]:
chat_history